In [1]:
import re

def price_filter_with_custom(query):
    """사용자 의도 분석을 통한 필터링"""

    filter_dict = {}

    # 가격 필터링을 위한 정규표현식 패턴
    # 1) N만원 이상 | N만원 부터
    price_pattern = r'(\d+)만원\s*(이상|부터)'
    price_match = re.search(price_pattern, query)
    
    if price_match:
        # SALE_PRC 키가 없으면 생성
        if "SALE_PRC" not in filter_dict:
            filter_dict["SALE_PRC"] = {}
        # 만원 단위를 원 단위로 변환 (예: 120만원 -> 1200000원)
        filter_dict["SALE_PRC"]["$gte"] = int(price_match.group(1)) * 10000
    
    # 2) N만원 이하 | N만원 까지
    price_pattern_lte = r'(\d+)만원\s*(이하|까지)'
    price_match_lte = re.search(price_pattern_lte, query)
    
    if price_match_lte:
        if "SALE_PRC" not in filter_dict:
            filter_dict["SALE_PRC"] = {}
        filter_dict["SALE_PRC"]["$lte"] = int(price_match_lte.group(1)) * 10000

    # 3) N만원 대
    price_pattern_range = r'(\d+)만원\s*대'
    price_match_range = re.search(price_pattern_range, query)
    
    if price_match_range:
        if "SALE_PRC" not in filter_dict:
            filter_dict["SALE_PRC"] = {}
        base_price = int(price_match_range.group(1)) * 10000
        filter_dict["SALE_PRC"]["$gte"] = base_price
        filter_dict["SALE_PRC"]["$lt"] = base_price * 2

    return filter_dict


In [9]:
import re

def price_filter_with_llm(intent):
    filter_dict = {}
    # 가격
    if "SALE_PRC" not in filter_dict:
        filter_dict["SALE_PRC"] = {}
    filter_dict["SALE_PRC"]["$gte"] = intent["SALE_PRC_GTE"]
    filter_dict["SALE_PRC"]["$lte"] = intent["SALE_PRC_LTE"]
    return filter_dict

def brand_filter_with_llm(intent):
    filter_dict = {}
    # 브랜드
    filter_dict["BRND_NM"] = intent["BRND_NM"]
    return filter_dict

def artc_filter_with_llm(intent):
    filter_dict = {}
    # 품목
    filter_dict["ARTC_NM"] = intent["ARTC_NM"]
    return filter_dict

def category_filter_with_llm(intent):
    filter_dict = {}
    # 카테고리
    filter_dict["CATEGORY_NM"] = intent["CATEGORY_NM"]
    return filter_dict

def features_filter_with_llm(intent):
    filter_dict = {}
    # 주요 기능 및 특징
    filter_dict["FEATURES"] = intent["FEATURES"]
    return filter_dict

In [10]:
# 의도 기반 필터링 함수 선언
def intent_based_filtering(query, intent):
    """사용자 의도 분석을 통한 필터링"""

    filter_dict = {}

    # 가격 필터
    filter_dict.update(price_filter_with_custom(query) or price_filter_with_llm(intent))

    # 브랜드 필터
    filter_dict.update(brand_filter_with_llm(intent))

    # 카테고리 필터
    filter_dict.update(category_filter_with_llm(intent))

    # 품목 필터
    filter_dict.update(artc_filter_with_llm(intent))

    # 주요 기능 및 특징 필터
    filter_dict.update(features_filter_with_llm(intent))

    return filter_dict

In [11]:
query = '5만원 이상 블루투스 마우스'
intent = {'SALE_PRC_GTE': 50000, 'SALE_PRC_LTE': 10000000, 'BRND_NM': '', 'ARTC_NM': '마우스', 'CATEGORY_NM': '컴퓨터 주변기기', 'FEATURES': '블루투스'}
print(intent_based_filtering(query, intent))

query = '4인 가족 냉장고 추천'
intent = {'SALE_PRC_GTE': 0, 'SALE_PRC_LTE': 10000000, 'BRND_NM': '', 'ARTC_NM': '냉장고', 'CATEGORY_NM': '가전', 'FEATURES': '4인 가족용'}
print(intent_based_filtering(query, intent))

{'SALE_PRC': {'$gte': 50000}, 'BRND_NM': '', 'CATEGORY_NM': '컴퓨터 주변기기', 'ARTC_NM': '마우스', 'FEATURES': '블루투스'}
{'SALE_PRC': {'$gte': 0, '$lte': 10000000}, 'BRND_NM': '', 'CATEGORY_NM': '가전', 'ARTC_NM': '냉장고', 'FEATURES': '4인 가족용'}
